In [1]:
import os
import shutil
import sys

In [33]:
# stories = {}
# source_folder = "./clean_dataset"

# dir_paths = []
# original_stories = []
# story_names = []

# for dir_name in os.listdir(source_folder):

#     if not dir_name.startswith(".DS_Store"):
#         dir_path = os.path.join(source_folder, dir_name)
#         dir_paths.append(dir_path)
#         for dir_path in dir_paths:
#             for story_name in os.listdir(dir_path):
#                 if not story_name.startswith(".DS_Store"):
#                     story_names.append(story_name)
#                     story_path = os.path.join(dir_path, story_name)
#                     file_path = story_path + "/story.txt"
#                     with open(file_path, 'r') as file:
#                         file_content = file.read()
#                     original_stories.append(file_content)

# stories["original_story"] = original_stories
# stories["story_name"] = story_names


In [1]:
import nltk
import sys
import tqdm
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'
os.environ['TORCH_USE_CUDA_DSA'] = '1'
from nltk.tokenize import TextTilingTokenizer
nltk.download('punkt') 
nltk.download('stopwords')
from transformers import pipeline
from collections import defaultdict
ttt = TextTilingTokenizer(w = 10, k=5)
device = "cuda:2"

def clean_segments(segments):
    clean_segment = []
    for segment in segments:
        if len(segment.strip())>10:
            clean_segment.append(segment)
    return clean_segment


def generateSegments(df):
    print("Cleaning segments")
    works = 0
    not_works = 0
    all_segments = []

    for index, row in tqdm.tqdm(df.iterrows()):
    
        story = row["original_story"]
        try:
            segments = ttt.tokenize(story)
            all_segments.append(clean_segments(segments))
            works = works+1                
            
        except:
            not_works = not_works + 1
            all_segments.append([])
            
    print("segmentation works", works)
    print("segmentation doesn't work", not_works)
    
    df["segmented_story"] = all_segments
    return df


def generateSummaries(df, summarizer_model_id):
    
    pipe = pipeline("summarization", model=summarizer_model_id, max_length=20, device = device)
    all_summaries = []
    
    for index, row in tqdm.tqdm(df.iterrows()):
        summaries = []
        segments = row["segmented_story"]
        for segment in segments:
            try:
                summary = pipe(segment)
                summaries.append(summary[0]['summary_text'])
            except Exception as e:
                print(f"An error occurred with model {summarizer_model_id}: {e}")
                summaries.append(segment)
                #append original segment in case of error
        all_summaries.append(summaries)
        
    df["summarized_story"] = all_summaries
    return df

[nltk_data] Downloading package punkt to /home/aditi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/aditi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import os
from utils import read_data
from image_generation import generate_images
os.environ['CUDA_VISIBLE_DEVICES']='2'
import nltk
nltk.download('punkt')  # For tokenization
nltk.download('stopwords')
from nltk.tokenize import TextTilingTokenizer

data_folder = "../data/mini_dataset_storygen"
images_out_dir = "../generated_images/baseline2"
df = read_data(data_folder)

df = generateSegments(df)

summarizer_model_id = "sshleifer/distilbart-xsum-12-6"


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to /home/aditi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/aditi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Cleaning segments


77it [00:25,  3.00it/s]

segmentation works 64
segmentation doesn't work 13


In [3]:
df = generateSummaries(df, summarizer_model_id)

df.to_csv(images_out_dir+"/df_summary.csv")

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [6]:
#generate segment wise data for baseline
# fairy_count = 0
# story_count = 0
# destination_folder = "../data/mini_dataset_baseline_segment_wise/"


# for index,row in df.iterrows():
#     if row["story_type"] == "fairy_tale" and fairy_count<25:
#         if len(row["segmented_story"]) > 0:
#             segments = row["segmented_story"]
#             fairy_count+=1
#             for idx, segment in enumerate(segments):
#                 file_name = row["story_name"] + "_segment_" + str(idx)
#                 with open(destination_folder+row["story_type"]+"/"+file_name+".txt", "w") as file:
#                     file.write(segment)
        
#     elif row["story_type"] == "short_story" and story_count<25:
#         if len(row["segmented_story"]) > 0:
#             segments = row["segmented_story"]
#             story_count+=1
#             for idx, segment in enumerate(segments):
#                 file_name = row["story_name"] + "_segment_" + str(idx)
#                 with open(destination_folder+row["story_type"]+"/"+file_name+".txt", "w") as file:
#                     file.write(segment)
#     else:
#         continue
    

In [ ]:
fairy_tales_folder = "../evaluation_data/baseline1/experiment_samit_fairy_tale_segmented"
short_story_folder = "../evaluation_data/baseline1/experiment_samit_short_story_segmented"